In [1]:
# -*- coding: utf-8
# Abraji (https://www.abraji.org.br)
# Reinaldo Chaves (reinaldo@abraji.org.br)
# Programa 01 do curso Como usar Python para descobrir uma rede de interesses
#

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

# Indústria Farmacêutica

## Procura empresas brasileiras e seus sócios 

Gerou estas reportagens: 
    
https://reporterbrasil.org.br/2019/02/industria-farmaceutica-ganha-assento-e-amplia-lobby-no-novo-congresso/
    
https://reporterbrasil.org.br/2019/02/executivo-conta-como-ganhou-subgabinete-no-senado-apos-investir-r-15-milhao-em-campanha/

https://brasil.elpais.com/brasil/2019/02/19/politica/1550534951_105635.html
    
https://brasil.elpais.com/brasil/2019/02/19/politica/1550580934_709058.html


1 - Primeiro foram procurados quais são as CNAEs (Classificação Nacional de Atividades Econômicas) das maiores empresas do setor em faturamento

57.507.378/0003-65 - EMS

02.932.074/0001-91 - Hypera

02.685.377/0001-57 - Sanofi

56994502000130 - Novartis

60.659.463/0001-91 - Aché

61.190.096/0001-92 - Eurofarma

60.397.775/0001-74 - Taked Pharma

18459628000115 - Bayer

17.159.229/0001-76 - Teuto Brasileiro

33.247.743/0001-10 - GSK

O resultado foi:

2121-1/01 Fabricação de medicamentos alopáticos para uso humano  

4771-7/01 Comércio varejista de produtos farmacêuticos, sem manipulação de fórmulas 

21.21-1-03 - Fabricação de medicamentos fitoterápicos para uso humano 

46.44-3-01 - Comércio atacadista de medicamentos e drogas de uso humano 

21.10-6-00 - Fabricação de produtos farmoquímicos 

21.21-1-02 - Fabricação de medicamentos homeopáticos para uso humano

Como são empresas referência, então esses CNAEs foram usados para procurar todas do setor farmacêutica no Brasil

Os CNAEs em geral podem ser vistos neste site: https://concla.ibge.gov.br/busca-online-cnae.html

Ou https://servicodados.ibge.gov.br/api/docs/cnae?versao=2

2 - Foi escrita uma rotina então para procurar os CNPJs que tivessem os CNAEs - no arquivo de empresas da Receita Federal

A fonte é: http://receita.economia.gov.br/orientacao/tributaria/cadastros/cadastro-nacional-de-pessoas-juridicas-cnpj/dados-publicos-cnpj

In [2]:
# Delimita o arquivo em chunks, em 100.000 divisões
# Aponta qual é o separador de informações
# Dá novos nomes para colunas
# Indica para ignorar o cabeçalho
# Define quais são os tipos das colunas
# Separa apenas as colunas de interesse
# Aponta o decimal usado

In [3]:
TextFileReader = pd.read_csv('dados/empresa.csv',\
                             chunksize=100000,\
                             sep=',',\
                             names=['cnpj',
                                    'identificador_matrizfilial',
                                    'razao_social',
                                    'nome_fantasia', 
                                    'situacao_cadastral',
                                    'data_situacao_cadastral',
                                    'motivo_situacao_cadastral', 
                                    'nome_cidade_exterior',
                                    'codigo_natureza_juridica',
                                    'data_inicio_atividade',
                                    'cnae_fiscal',
                                    'descricao_tipo_logradouro',
                                    'logradouro', 
                                    'numero', 
                                    'complemento', 
                                    'bairro', 
                                    'cep', 
                                    'uf', 
                                    'codigo_municipio', 
                                    'municipio', 
                                    'ddd_telefone_1',
                                    'ddd_telefone_2', 
                                    'ddd_fax', 
                                    'qualificacao_do_responsavel', 
                                    'capital_social', 
                                    'porte', 
                                    'opcao_pelo_simples', 
                                    'data_opcao_pelo_simples', 
                                    'data_exclusao_do_simples', 
                                    'opcao_pelo_mei', 
                                    'situacao_especial', 
                                    'data_situacao_especial'],\
                             header=None,\
                             dtype={'cnpj': str,
                                    'identificador_matrizfilial': str,
                                    'razao_social': str,
                                    'nome_fantasia': str, 
                                    'situacao_cadastral': str,
                                    'data_situacao_cadastral': str,
                                    'motivo_situacao_cadastral': str, 
                                    'nome_cidade_exterior': str,
                                    'codigo_natureza_juridica': str,
                                    'data_inicio_atividade': str,
                                    'cnae_fiscal': str,
                                    'descricao_tipo_logradouro': str,
                                    'logradouro': str, 
                                    'numero': str, 
                                    'complemento': str, 
                                    'bairro': str, 
                                    'cep': str, 
                                    'uf': str, 
                                    'codigo_municipio': str, 
                                    'municipio': str, 
                                    'ddd_telefone_1': str,
                                    'ddd_telefone_2': str, 
                                    'ddd_fax': str, 
                                    'qualificacao_do_responsavel': str, 
                                    'capital_social': str, 
                                    'porte': str, 
                                    'opcao_pelo_simples': str, 
                                    'data_opcao_pelo_simples': str, 
                                    'data_exclusao_do_simples': str, 
                                    'opcao_pelo_mei': str, 
                                    'situacao_especial': str, 
                                    'data_situacao_especial': str},\
                             usecols=['cnpj',
                                      'identificador_matrizfilial',
                                      'razao_social',
                                      'nome_fantasia', 
                                      'situacao_cadastral',
                                      'data_situacao_cadastral',
                                      'motivo_situacao_cadastral', 
                                      'nome_cidade_exterior',
                                      'codigo_natureza_juridica',
                                      'data_inicio_atividade',
                                      'cnae_fiscal',
                                      'uf', 
                                      'municipio', 
                                      'qualificacao_do_responsavel', 
                                      'capital_social'
                                    ],\
                            decimal=',') 

In [4]:
now = datetime.now()
print("A extração começou às: ", now)

# Estabelece contagem de vezes
conta = 0

# Iteração em cada chunk
for df in TextFileReader:
        
    # Procura as informações de interesse
    # Situacao cadastral 2 são empresas Ativas
    identificador = df['cnae_fiscal'].isin(['2121101', 
                                  '4771701',
                                  '2121103',
                                  '4644301',
                                  '2110600',
                                  '2121102']) & (df['situacao_cadastral'] == '2')
    nome = df[identificador]
    
    # Testa se encontrou algo
    if nome.empty is False:
        # Se for a primeira vez cria o dataframe df_procura1
        if conta == 0:
            df_procura1 = nome
        # Se não for a primeira vez faz append no dataframe df_procura1
        else:
            df_procura1 = df_procura1.append(nome)
    
        conta = conta + 1
    
now = datetime.now()
print("A extração terminou às: ", now)

A extração começou às:  2019-06-22 12:56:56.451549
A extração terminou às:  2019-06-22 13:00:42.786024


In [5]:
df_procura1.head()

,cnpj,identificador_matrizfilial,razao_social,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,codigo_natureza_juridica,data_inicio_atividade,cnae_fiscal,uf,municipio,qualificacao_do_responsavel,capital_social
9103,00001108000107,1,NOELI SEILA BOIAN,DROGARIA SAO JUDAS TADEU,2,2004-02-20,0,NaN,2135,1994-05-05,4771701,SP,GUARARAPES,50,0
10101,00002034000115,1,FABIO ZATTE,DROGARIA SAO LUCAS,2,2005-06-24,0,NaN,2135,1994-05-13,4771701,SP,TUPI PAULISTA,50,1000000
11007,00002903000101,1,DROGARIA SANTA RITA DE CANDIDO MOTA LTDA,NaN,2,2004-01-23,0,NaN,2062,1994-05-12,4771701,SP,CANDIDO MOTA,49,200000
11388,00003230000104,1,FOLTRAN & NATALIO LTDA,FARMACIA BELA VISTA,2,2000-12-01,0,NaN,2062,1994-05-12,4771701,SP,PIRACICABA,49,0
13240,00005012000109,1,ANALICE SOARES LEITE DE ARAUJO,NaN,2,2006-10-13,0,NaN,2135,1994-05-16,4771701,SP,ANGATUBA,50,1000000


In [5]:
df_procura1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118639 entries, 9103 to 40754222
Data columns (total 15 columns):
cnpj                           118639 non-null object
identificador_matrizfilial     118639 non-null object
razao_social                   118639 non-null object
nome_fantasia                  89674 non-null object
situacao_cadastral             118639 non-null object
data_situacao_cadastral        118639 non-null object
motivo_situacao_cadastral      118639 non-null object
nome_cidade_exterior           42 non-null object
codigo_natureza_juridica       118639 non-null object
data_inicio_atividade          118639 non-null object
cnae_fiscal                    118639 non-null object
uf                             118639 non-null object
municipio                      118639 non-null object
qualificacao_do_responsavel    118639 non-null object
capital_social                 118639 non-null object
dtypes: object(15)
memory usage: 14.5+ MB


In [6]:
# Salva o resultado

In [7]:
df_procura1.to_csv('resultados/empresas_farm_cnae_principal.csv', index=False)

3 - A Receita tem um arquivo também com os CNPJs divididos pelo CNAEs secundários. 

Às vezes as empresas cadastram suas atividades mais diretas no CNAE secundário. Não é o caso da indústria farmacêutica

Aqui foi feito para fins de exemplo

Foi escrita a mesma rotina para este arquivo

In [8]:
TextFileReader = pd.read_csv('dados/cnae-secundaria.csv',\
                             chunksize=100000,\
                             sep=',',\
                             names=['cnpj',
                                    'cnae_secundaria'
                                    ],\
                             header=None,\
                             dtype={'cnpj': str,
                                    'cnae_secundaria': str},
                             usecols=['cnpj',
                                      'cnae_secundaria']) 

In [9]:
now = datetime.now()
print("A extração começou às: ", now)

# Estabelece contagem de vezes
conta = 0

# Iteração em cada chunk
for df in TextFileReader:
        
    # Procura as informações de interesse    
    identificador = df['cnae_secundaria'].isin(['2121101', 
                                  '4771701',
                                  '2121103',
                                  '4644301',
                                  '2110600',
                                  '2121102'])
    nome = df[identificador]
    
    # Testa se encontrou algo
    if nome.empty is False:
        # Se for a primeira vez cria o dataframe df_procura1
        if conta == 0:
            df_procura2 = nome
        # Se não for a primeira vez faz append no dataframe df_procura1
        else:
            df_procura2 = df_procura2.append(nome)
    
        conta = conta + 1
    
now = datetime.now()
print("A extração terminou às: ", now)

A extração começou às:  2019-06-22 13:01:10.224689
A extração terminou às:  2019-06-22 13:01:40.302683


In [10]:
df_procura2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45252 entries, 6758 to 49321774
Data columns (total 2 columns):
cnpj               45252 non-null object
cnae_secundaria    45252 non-null object
dtypes: object(2)
memory usage: 1.0+ MB


In [11]:
df_procura2.to_csv('resultados/empresas_farm_cnae_secundaria.csv', index=False)

O arquivo do CNAE principal precisa ser carregado apenas se o usuário encerrar a sessão

Se continuar na mesma seção pode pular esta parte

In [18]:
df_procura1 = pd.read_csv("resultados/empresas_farm_cnae_principal.csv", converters={'cnpj': lambda x: str(x),
                                                                              'cnae_fiscal': lambda x: str(x)},\
                  decimal=',')

In [19]:
df_procura1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 118639 entries, 0 to 118638
Data columns (total 15 columns):
cnpj                           118639 non-null object
identificador_matrizfilial     118639 non-null int64
razao_social                   118639 non-null object
nome_fantasia                  89674 non-null object
situacao_cadastral             118639 non-null int64
data_situacao_cadastral        118639 non-null object
motivo_situacao_cadastral      118639 non-null int64
nome_cidade_exterior           42 non-null object
codigo_natureza_juridica       118639 non-null int64
data_inicio_atividade          118639 non-null object
cnae_fiscal                    118639 non-null object
uf                             118639 non-null object
municipio                      118639 non-null object
qualificacao_do_responsavel    118639 non-null int64
capital_social                 118639 non-null object
dtypes: int64(5), object(10)
memory usage: 13.6+ MB


In [21]:
df_procura1.reset_index().head()

,index,cnpj,identificador_matrizfilial,razao_social,nome_fantasia,situacao_cadastral,data_situacao_cadastral,motivo_situacao_cadastral,nome_cidade_exterior,codigo_natureza_juridica,data_inicio_atividade,cnae_fiscal,uf,municipio,qualificacao_do_responsavel,capital_social
0,0,00001108000107,1,NOELI SEILA BOIAN,DROGARIA SAO JUDAS TADEU,2,2004-02-20,0,NaN,2135,1994-05-05,4771701,SP,GUARARAPES,50,0.00
1,1,00002034000115,1,FABIO ZATTE,DROGARIA SAO LUCAS,2,2005-06-24,0,NaN,2135,1994-05-13,4771701,SP,TUPI PAULISTA,50,10000.00
2,2,00002903000101,1,DROGARIA SANTA RITA DE CANDIDO MOTA LTDA,NaN,2,2004-01-23,0,NaN,2062,1994-05-12,4771701,SP,CANDIDO MOTA,49,2000.00
3,3,00003230000104,1,FOLTRAN & NATALIO LTDA,FARMACIA BELA VISTA,2,2000-12-01,0,NaN,2062,1994-05-12,4771701,SP,PIRACICABA,49,0.00
4,4,00005012000109,1,ANALICE SOARES LEITE DE ARAUJO,NaN,2,2006-10-13,0,NaN,2135,1994-05-16,4771701,SP,ANGATUBA,50,10000.00


Diminue o número de colunas, só as que interessam

In [12]:
farma_final = df_procura1.drop(['motivo_situacao_cadastral',
                        ] , axis=1)

In [13]:
farma_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 118639 entries, 9103 to 40754222
Data columns (total 14 columns):
cnpj                           118639 non-null object
identificador_matrizfilial     118639 non-null object
razao_social                   118639 non-null object
nome_fantasia                  89674 non-null object
situacao_cadastral             118639 non-null object
data_situacao_cadastral        118639 non-null object
nome_cidade_exterior           42 non-null object
codigo_natureza_juridica       118639 non-null object
data_inicio_atividade          118639 non-null object
cnae_fiscal                    118639 non-null object
uf                             118639 non-null object
municipio                      118639 non-null object
qualificacao_do_responsavel    118639 non-null object
capital_social                 118639 non-null object
dtypes: object(14)
memory usage: 13.6+ MB


4 - Agora é o momento de preencher o dataframe de empresas com os nomes e CPFs de sócios

A Receita tem um outro arquivo apenas com esses dados

Os dados de CPF são gravados neste formato: 3 asterisicos + seis números + dois asteriscos

In [14]:
TextFileReader = pd.read_csv('dados/socio.csv',\
                             chunksize=100000,\
                             sep=',',\
                             names=['cnpj',
                                    'identificador_de_socio',
                                    'nome_socio',
                                    'cnpj_cpf_do_socio',
                                    'codigo_qualificacao_socio',
                                    'percentual_capital_social',
                                    'data_entrada_sociedade',
                                    'cpf_representante_legal',
                                    'nome_representante_legal',
                                    'codigo_qualificacao_representante_legal'],\
                             header=None,\
                             dtype={'cnpj': str,
                                    'identificador_de_socio': str,
                                    'nome_socio': str,
                                    'cnpj_cpf_do_socio': str,
                                    'codigo_qualificacao_socio': str,
                                    'percentual_capital_social': str,
                                    'data_entrada_sociedade': str,
                                    'cpf_representante_legal': str,
                                    'nome_representante_legal': str,
                                    'codigo_qualificacao_representante_legal': str
},\
                             usecols=['cnpj',
                                      'nome_socio',
                                      'cnpj_cpf_do_socio',
                                      'percentual_capital_social',
                                      'codigo_qualificacao_socio',
                                      'cpf_representante_legal',
                                      'nome_representante_legal'],
                             decimal=',')  

In [15]:
now = datetime.now()
print("A extração começou às: ", now)

conta = 0
for df in TextFileReader:

    # Caso deseja-se pegar o CNPJ raiz poderia criar uma nova coluna
    #df['cnpj_raiz'] = df.cnpj.str[:8]
    
    # Faz merge dos dataframes pelo número do CNPJ
    # Se por algum motivo no dataframe original existirem CNPJ duplicados faz um drop
    farm_encontrados = pd.merge(farma_final.drop_duplicates('cnpj'), df, left_on='cnpj', right_on='cnpj')
    
    if farm_encontrados.empty is False:
        if conta == 0:
            df_procura3 = farm_encontrados
        else:
            df_procura3 = df_procura3.append(farm_encontrados)
    
        conta = conta + 1
        
now = datetime.now()
print("A extração terminou às: ", now)

A extração começou às:  2019-06-22 13:02:06.183891
A extração terminou às:  2019-06-22 13:03:53.724043


In [16]:
df_procura3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 201085 entries, 0 to 479
Data columns (total 20 columns):
cnpj                           201085 non-null object
identificador_matrizfilial     201085 non-null object
razao_social                   201085 non-null object
nome_fantasia                  141779 non-null object
situacao_cadastral             201085 non-null object
data_situacao_cadastral        201085 non-null object
nome_cidade_exterior           11 non-null object
codigo_natureza_juridica       201085 non-null object
data_inicio_atividade          201085 non-null object
cnae_fiscal                    201085 non-null object
uf                             201085 non-null object
municipio                      201085 non-null object
qualificacao_do_responsavel    201085 non-null object
capital_social                 201085 non-null object
nome_socio                     201085 non-null object
cnpj_cpf_do_socio              201085 non-null object
codigo_qualificacao_socio      2

In [17]:
df_procura3.to_csv('resultados/empresas_farm_com_socios.csv', index=False)

### Atenção: os campos capital_social_da_empresa e percentual_capital_social a Receita ainda fornece no arquivo com erros de formatação. É necessários checar eles no site antes de usar - http://www.receita.fazenda.gov.br/PessoaJuridica/CNPJ/cnpjreva/Cnpjreva_Solicitacao.asp